In [ ]:
# %load train.py
import tensorflow as tf
import numpy as np
import os
import cv2
import dense_net

import anchor_op
from xml_op import *
%matplotlib inline

anchor_num = 3
a_num = 1
ratio_num = 3
stride=np.array([7,14,28,56])
anchor_size=np.array([64,32,16,8])
ratio = np.array([[1,1],[1,2],[2,1]])

rpn_ckpts = './checkpoint2_dir'
rpn_adam_meta = './checkpoint2_dir/MyModel'

roi_ckpts = './roi_ck2'
roi_adam_meta = './roi_ck2/MyModel'
   

def check_enviroment(path):
    if(not os.path.exists(path)):
        os.mkdir(path)
        print('create dir')
    else:
        print('already exist')
        
def show_img(img):
    plt.imshow(img)
    plt.show()

def cal_l1_loss(real_box_dx,pre_box,is_real_box,ll1,ll2,ll3):
    j1 = tf.add(real_box_dx,(-1)*pre_box)
    j2 = tf.cast(tf.less(tf.abs(j1),ll1),tf.float32)
    j3 = is_real_box*((j2)*(ll2*j1*j1)+(1.0-j2)*(tf.abs(j1)-ll3))
    j4 = tf.reduce_sum(j3)
    return j4,j3

def region_proposal(anchor_ut,sort_id,sort_score,dx_box_loc,r_box_loc):
    rpn_anchor = anchor_ut.get_best_anchor(sort_id,sort_score,dx_box_loc)
    #print(rpn_anchor)
    rpn_anchor_index = anchor_ut.py_cpu_nms(rpn_anchor)
    dx_box_loc = rpn_anchor[rpn_anchor_index,0:4]
    #print(dx_box_loc)
    #print(dx_box_loc.shape)
    fg_bg_check,real_fg_bg_value,box_index,is_anchor = anchor_ut.collect_roi_fg_bg(dx_box_loc,r_box_loc,pos_iou=0.5,neg_iou=0.1,max_anchor=128)
    #print(fg_bg_check)
    #print(real_fg_bg_value)
    #print(is_anchor)
    rpn_box = np.zeros([rpn_anchor_index.shape[0],6])
    for i in range(fg_bg_check.shape[1]):
        if(fg_bg_check[0][i]!=0):
            x1,y1,x2,y2 = dx_box_loc[i]
            rpn_box[i,:4] = x1,y1,x2,y2
            
            w = x2-x1
            h = y2-y1
            feature_num = 3+int(np.log2((np.sqrt(w*h)/224)))
            if(feature_num<0):
                feature_num = 0
            if(real_fg_bg_value[0][i][0]==1):
                rpn_box[i][4] = feature_num
                rpn_box[i][5] = 0    
            else:
                rpn_box[i][4] = feature_num
                rpn_box[i][5] = 1
            
    #print(rpn_box[:,5])
    real_fg_bg_value = np.squeeze(real_fg_bg_value)
    fg_bg_check = np.squeeze(fg_bg_check)
    #print(real_fg_bg_value.shape)
    return rpn_box,fg_bg_check,real_fg_bg_value,box_index,is_anchor

def rpn_train():
    train_data = read_xml('./outputs')
    #train_data = read_xml('./xml_test')
    '''
    img = cv2.imread('test.jpg')
    r_img = np.reshape(img,(-1,224,224,3))
    r_box = np.reshape(np.array([[78,54,148,173]],dtype=np.float),(-1,4))
    '''
    ds = dense_net.Dense_net(224,3,10,24,0.5)
    x = ds.x
    
    pre_fg,pre_fg_score = ds.all_fg,ds.all_fg_score
    pre_box_dx = ds.all_box
    
    sort_fg_index,sort_fg_score = ds.sort_fg_score()
    
    real_fg = tf.placeholder(tf.float32,[None,None,2])
    is_real_fg = tf.placeholder(tf.float32,[None,None])
    fg_num = tf.placeholder(tf.float32)
    
    real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    is_real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    box_num = tf.placeholder(tf.float32)
    
    fg_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
                                    labels=tf.argmax(real_fg,axis=2),logits=pre_fg)

    fg_loss = tf.reduce_sum(tf.multiply(fg_loss,is_real_fg))
    fg_loss = fg_loss/fg_num
    box_loss,j1 = cal_l1_loss(real_box_dx,pre_box_dx,is_real_box_dx,1.0,0.5,0.5)
    box_loss = (box_loss/box_num)*5
    box_loss = tf.clip_by_value(box_loss,1e-10,box_loss)
    rpn_total_loss = fg_loss+box_loss
    
    global_step = tf.get_variable('global_step', [], dtype=tf.int32,
                                  initializer=tf.constant_initializer(0), trainable=False)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.name_scope('opt'):
        #opt = tf.train.AdamOptimizer(1e-3,name='optimizer')
        #opt = tf.train.GradientDescentOptimizer(5e-3,name='optimizer')
        opt = tf.train.MomentumOptimizer(1e-4,0.9,name='optimizer')
        with tf.control_dependencies(update_ops):
            grads = opt.compute_gradients(rpn_total_loss)
            train_op = opt.apply_gradients(grads, global_step=global_step)
        #tf.summary.scalar('opti', opt)
    
    anchor_ut = anchor_op.anchor_util(stride,anchor_size)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        ########  save
        var_list = tf.trainable_variables()
        if global_step is not None:
            var_list.append(global_step)
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list,max_to_keep=5)
        ########
        
        if tf.train.latest_checkpoint(rpn_ckpts) is not None:
            saver.restore(sess, tf.train.latest_checkpoint(rpn_ckpts))
            #saver = tf.train.import_meta_graph('./checkpoint1_dir/MyModel-3250.meta')
        else:
            assert 'can not find checkpoint folder path!'
        
        
        for i in range(1000):
            print('-----')
            
            ba_x = i%15
            print(train_data[ba_x])
            img,r_box = decode_xml(train_data[ba_x])
            r_img = img[np.newaxis,:,:,:]
            
            #print(r_img.shape)
            
            pf_value,pf_score,dx_box,sort_id,sort_score,td = sess.run([pre_fg,pre_fg_score,pre_box_dx,
                                                 sort_fg_index,sort_fg_score,ds.dense4],feed_dict={x:r_img})
    
            dx_box_loc = anchor_ut.get_dx_box(dx_box[0])
            
            fg_bg_check,real_fg_bg_value,box_index,is_anchor = anchor_ut.collect_fg_bg(dx_box_loc,r_box)
            
            dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(dx_box,box_index,real_fg_bg_value,r_box)
                          
            
            fg_ls,box_ls,fg_rank,_= sess.run([fg_loss,box_loss,sort_fg_index,train_op],feed_dict={x:r_img,
                                               real_box_dx:dx_anchor_value,is_real_box_dx:is_dx_box,box_num:is_box,
                                               real_fg:real_fg_bg_value,is_real_fg:fg_bg_check,fg_num:is_anchor
                                               })
                    
            if(i%1==0):
                for j in range(pf_score.shape[1]):
                    if(real_fg_bg_value[0][j][0]==1):
                        rank = 0
                        for k in range(fg_rank.shape[0]):
                            if(fg_rank[k]==j):
                                rank = k
                        #print('high',pf_score[0][fg_rank[0]])
                        #print('low',pf_score[0][fg_rank[-1]])
                        print('fg_score',pf_score[0][j],'rank',rank,' id',j)
                        print(fg_ls,box_ls)
                        
            if(i%50==0 and i!=0):
                saver.save(sess,rpn_adam_meta,global_step=i) 
                
def roi_train():
    train_data = read_xml('./outputs')
    '''
    img = cv2.imread('test.jpg')
    r_img = np.reshape(img,(-1,224,224,3))
    r_box = np.reshape(np.array([[78,54,148,173],[28,12,64,99]],dtype=np.float),(-1,4))
    '''
    ds = dense_net.Dense_net(224,3,10,24,0.5)
    x = ds.x
    
    pre_fg,pre_fg_score = ds.all_fg,ds.all_fg_score
    pre_box_dx = ds.all_box
    
    sort_fg_index,sort_fg_score = ds.sort_fg_score()
       
    roi_cls_score,roi_cls_score_p,roi_box_dx = ds.cls_score,ds.cls_score_p,ds.final_box
    #print(roi_cls_score)
    roi_feature_map = ds.roi_feature_map
    roi_cls = tf.placeholder(tf.float32,[None,2])
    roi_is_fg = tf.placeholder(tf.float32,[None])
    roi_fg_num = tf.placeholder(tf.float32)
    '''
    rpn_boxes = tf.placeholder(tf.float32,[None,4])
    rpn_num = tf.placeholder(tf.int32)
    roi_feature_2 = ds.roi_pooling_2(rpn_boxes,rpn_num)
    print(roi_feature_2)
    '''
    real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    is_real_box_dx = tf.placeholder(tf.float32,[None,None,4])
    box_num = tf.placeholder(tf.float32)
    
    box_loss,j1 = cal_l1_loss(real_box_dx,ds.final_box,is_real_box_dx,1.0,0.5,0.5)
    box_loss = (box_loss/box_num)
    box_loss = tf.clip_by_value(box_loss,1e-10,box_loss)


    cls_loss =  tf.nn.sparse_softmax_cross_entropy_with_logits(
                                    labels=tf.argmax(roi_cls,axis=1),logits=roi_cls_score)
    cls_loss = tf.reduce_sum(tf.multiply(cls_loss,roi_is_fg))
    cls_loss = cls_loss/roi_fg_num
    
    roi_total_loss = cls_loss+box_loss
    
    global_step = tf.get_variable('global_step', [], dtype=tf.int32,
                                  initializer=tf.constant_initializer(0), trainable=False)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.name_scope('roi_opt'):
        #opt = tf.train.AdamOptimizer(1e-3,name='optimizer')
        #opt = tf.train.GradientDescentOptimizer(5e-3,name='optimizer')
        opt = tf.train.MomentumOptimizer(1e-4,0.9,name='optimizer')
        with tf.control_dependencies(update_ops):
            grads = opt.compute_gradients(roi_total_loss)
            train_op = opt.apply_gradients(grads, global_step=global_step)
        #tf.summary.scalar('opti', opt)
    
    anchor_ut = anchor_op.anchor_util(stride,anchor_size)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        ########  saves
        var_list = tf.trainable_variables()
        if global_step is not None:
            var_list.append(global_step)
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list,max_to_keep=5)
        ########
        
        if tf.train.latest_checkpoint(rpn_ckpts) is not None:
            saver.restore(sess, tf.train.latest_checkpoint(rpn_ckpts))
        else:
            assert 'can not find checkpoint folder path!'
        
        
        for i in range(1000):
            print('-----')
            ba_x = i%15
            #print(train_data[ba_x])
            img,r_box = decode_xml(train_data[ba_x])
            r_img = img[np.newaxis,:,:,:]
            
            pf_value,pf_score,dx_box,sort_id,sort_score = sess.run([pre_fg,pre_fg_score,pre_box_dx,
                                                 sort_fg_index,sort_fg_score],feed_dict={x:r_img})
    
            dx_box_loc = anchor_ut.get_dx_box(dx_box[0])
            #print(dx_box_loc)
            if(i<=60):
                rpn_box,roi_fg_bg_check,roi_real_fg_bg_value,roi_box_index,roi_is_anchor = region_proposal(anchor_ut,sort_id,sort_score,dx_box_loc,r_box)

                roi_feature,cls = ds.roi_pooling(rpn_box)
                r_feature = sess.run(roi_feature,feed_dict={x:r_img})

                rcs,rcsp,rdx = sess.run([roi_cls_score,roi_cls_score_p,roi_box_dx],feed_dict={roi_feature_map:r_feature})
                print(rdx[0],rpn_box[0],roi_box_dx)

                dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(rpn_box[np.newaxis,:,:4],
                                                                        roi_box_index,roi_real_fg_bg_value[np.newaxis,:,:],r_box,rpn_box[:,:4])
            
            else:
                ####
                rpn_box = anchor_ut.region_proposal_2(sort_id,sort_score,dx_box_loc)
                roi_feature = ds.roi_pooling_1(rpn_box)
                r_feature = sess.run(roi_feature,feed_dict={x:r_img})
                rcs,rcsp,rdx = sess.run([roi_cls_score,roi_cls_score_p,roi_box_dx],feed_dict={roi_feature_map:r_feature})
                print(rdx[0],rpn_box[0])
                dx_box_loc2 = anchor_ut.get_dx_box(rdx,rpn_box[:,:4])
                roi_fg_bg_check,roi_real_fg_bg_value,roi_box_index,roi_is_anchor = anchor_ut.collect_roi_fg_bg(dx_box_loc2,r_box,pos_iou=0.5,neg_iou=0.1,max_anchor=128)
                dx_anchor_value,is_dx_box,is_box = anchor_ut.collect_anchor_dx(dx_box_loc2,
                                                                        roi_box_index,roi_real_fg_bg_value,r_box,rpn_box[:,:4])
                roi_real_fg_bg_value = np.reshape(roi_real_fg_bg_value,(-1,2))
                roi_fg_bg_check = roi_fg_bg_check[0]
                ####
            
            '''
            #print(rpn_box)
            #r_feature = sess.run(roi_feature,feed_dict={rpn_boxes:rpn_box[np.newaxis,:,:],rpn_num:rpn_box.shape[0]})
            r = sess.run(roi_feature_2,feed_dict={rpn_boxes:rpn_box[:,:4],rpn_num:rpn_box.shape[0],x:r_img})
            print(r.shape,r[0][0][0][0:10])
            #print(dsf)
            '''
            for j in range(roi_real_fg_bg_value.shape[0]):
                if(roi_real_fg_bg_value[j][0]==1):
                    print('is fg',rcsp[j])
                '''     
                    im = img.copy()
                    x1,y1,x2,y2 = rpn_box[j,:4]

                        cv2.rectangle(im,(int(x1),int(y1)),(int(x2),int(y2)), (0, 255, 0), 2)
                        show_img(im)
                else:
                    print(rcsp[j])
                    cv2.rectangle(im,(int(x1),int(y1)),(int(x2),int(y2)), (0, 255, 0), 2)
                    show_img(im)
                '''   
            
            #print(roi_is_anchor)
            #print(roi_real_fg_bg_value.shape,roi_fg_bg_check.shape,roi_is_anchor)
            #print(is_dx_box.shape,dx_anchor_value.shape,is_dx_box.shape,is_box)
            cls_ls,box_ls,_ = sess.run([cls_loss,box_loss,train_op],
                                        feed_dict={roi_cls:roi_real_fg_bg_value,roi_is_fg:roi_fg_bg_check,
                                           roi_fg_num:roi_is_anchor,roi_feature_map:r_feature,x:r_img,
                                           real_box_dx:dx_anchor_value,is_real_box_dx:is_dx_box,box_num:is_box})
            
            print(cls_ls,box_ls)
                        
            if(i%100==0 and i!=0):
                saver.save(sess,roi_adam_meta,global_step=i) 


if(__name__=='__main__'):
    #rpn_train()
    roi_train()

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Tensor("dense_net/resnext/max_pooling2d/MaxPool:0", shape=(?, 56, 56, 64), dtype=float32)
Tensor("dense_net/resnext/Relu_17:0", shape=(?, 56, 56, 64), dtype=float32)
Tensor("dense_net/resnext/Relu_34:0", shape=(?, 28, 28, 128), dtype=float32)
Tensor("dense_net/resnext/Relu_51:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("dense_net/resnext/Relu_68:0", shape=(?, 7, 7, 512), dtype=float32)
(<tf.Tensor 'concat:0' shape=(?, 12495, 2) dtype=float32>, <tf.Tensor 'concat_1:0' shape=(?, 12495, 2) dtype=float32>, <tf.Tensor 'concat_2:0' shape=(?, 12495, 4) dtype=float32>)
Tensor("flatten/Reshape:0", shape=(?, 12544), dtype=float32)
(?, 12495, 2)
INFO:tensorflow:Restoring parameters from ./checkpoint2_dir/MyModel-150
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
0.9615384615384616
0.9803921568627451
0.5173288250211328
0.9349593495934959
(4, 10)
(array([-0.96699893,  0.49325764, -0.1525035 ,  0.324655  ], dtype=float

(array([-0.01603042, -0.19174795, -0.30988416, -0.1672666 ], dtype=float32), array([151., 120., 199., 187.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.8323192 , 0.16768084], dtype=float32))
('is fg', array([0.90950716, 0.09049286], dtype=float32))
('is fg', array([0.6926705 , 0.30732945], dtype=float32))
('is fg', array([0.7794127 , 0.22058736], dtype=float32))
('is fg', array([0.8522938 , 0.14770626], dtype=float32))
(0.5851421, 0.19240817)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
0.9615384615384616
0.9803921568627451
0.5173288250211328
0.9349593495934959
(4, 10)
(array([ 0.39312127, -0.08695547, -0.16912684,  0.1099202 ], dtype=float32), array([ 87.,  35., 138., 106.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.87878793, 0.12121212], dtype=float32))
('is fg', array([0.8540716 , 0.14592837], dtype=float32))
('is fg', array([0.88772416, 0.11227585], dtype=float32))
('is fg', arra

(array([ 0.32490197,  0.16330884, -0.05141404,  0.2127503 ], dtype=float32), array([ 87.,  35., 138., 106.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.92814523, 0.07185483], dtype=float32))
('is fg', array([0.8959516 , 0.10404838], dtype=float32))
('is fg', array([0.9388205 , 0.06117947], dtype=float32))
('is fg', array([0.8397684 , 0.16023158], dtype=float32))
(0.46538547, 0.054185137)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
0.9652650822669104
1.0
0.9510657460222155
0.9660421545667447
0.9795918367346939
0.9459540161242161
(6, 13)
(array([ 0.0276326 ,  0.04409926,  0.23092076, -0.01282462], dtype=float32), array([150.,  37., 199., 104.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.812606  , 0.18739401], dtype=float32))
('is fg', array([0.9118994 , 0.08810057], dtype=float32))
('is fg', array([0.79684293, 0.20315707], dtype=float32))
('is fg', array([0.7863329 , 0.2

(array([-0.01013691,  0.02299865,  0.18265511, -0.00021379], dtype=float32), array([150.,  37., 199., 104.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.83858776, 0.1614123 ], dtype=float32))
('is fg', array([0.9343225 , 0.06567752], dtype=float32))
('is fg', array([0.80784583, 0.19215423], dtype=float32))
('is fg', array([0.8202613 , 0.17973875], dtype=float32))
('is fg', array([0.80117804, 0.19882198], dtype=float32))
('is fg', array([0.68326646, 0.3167335 ], dtype=float32))
(0.2702946, 0.045796398)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]
0.9493670886075949
(1, 25)
(array([ 0.00423066, -0.06462956, -0.04616095,  0.00173052], dtype=float32), array([120.,  65., 186., 156.,   2.,   0.]), <tf.Tensor 'dense_3/BiasAdd:0' shape=(?, 4) dtype=float32>)
('is fg', array([0.8892271 , 0.11077289], dtype=float32))
(0.32228047, 0.003985631)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 1

...

(array([ 0.01154715, -0.00271172,  0.01881251, -0.01245819], dtype=float32), array([122.,  66., 187., 155.,   2.]))
0.9846153846153847
0.9371091035441279
0.9775280898876404
0.9630996309963099
(4, 23)
('is fg', array([0.9730566 , 0.02694339], dtype=float32))
('is fg', array([0.9746436 , 0.02535645], dtype=float32))
('is fg', array([0.94452816, 0.05547182], dtype=float32))
('is fg', array([0.85757947, 0.14242058], dtype=float32))
(0.100087605, 0.00024770026)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
(array([-0.00059707,  0.00425393, -0.00836053, -0.01352861], dtype=float32), array([122.,  66., 186., 156.,   2.]))
0.9626671297100191
0.9777777777777777
(2, 33)
('is fg', array([0.985768  , 0.01423193], dtype=float32))
('is fg', array([0.94732505, 0.05267499], dtype=float32))
(0.059224054, 4.7715417e-05)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
(array([0.00844212, 0.004

(array([ 0.01139078,  0.01090258, -0.00209442,  0.00265707], dtype=float32), array([ 88.,  35., 137., 103.,   2.]))
0.9323131253678635
0.9463612641345317
0.9518701072774717
0.96
0.9795918367346939
(5, 5)
('is fg', array([0.9938397 , 0.00616033], dtype=float32))
('is fg', array([0.9922891 , 0.00771089], dtype=float32))
('is fg', array([0.98458374, 0.01541625], dtype=float32))
('is fg', array([0.9975096 , 0.00249035], dtype=float32))
('is fg', array([0.99600875, 0.00399126], dtype=float32))
(0.050432086, 0.00016943029)
-----
[ 0  1  2  3  4  5  6  7  8  9 10]
(array([0.00201683, 0.01650952, 0.02502752, 0.00922519], dtype=float32), array([ 87., 118., 138., 188.,   2.]))
0.8983739837398373
0.95361380798274
0.7866258295048494
0.9302325581395349
0.94896331738437
0.9478672985781991
0.9345794392523364
(7, 4)
('is fg', array([0.956672  , 0.04332799], dtype=float32))
('is fg', array([0.91840756, 0.08159249], dtype=float32))
('is fg', array([0.955988  , 0.04401203], dtype=float32))
('is fg', arra

(array([ 0.00756636,  0.01757484, -0.00410336, -0.02398516], dtype=float32), array([120.,  65., 186., 156.,   2.]))
0.9129993229519295
0.937046004842615
0.9491386382280558
0.9701492537313433
0.9305307579938322
(5, 21)
('is fg', array([0.98417735, 0.01582266], dtype=float32))
('is fg', array([0.9410901 , 0.05890986], dtype=float32))
('is fg', array([0.87169266, 0.12830736], dtype=float32))
('is fg', array([0.92402464, 0.0759754 ], dtype=float32))
('is fg', array([0.89053535, 0.10946466], dtype=float32))
(0.102815755, 0.00043941793)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
(array([-0.0145053 , -0.01685737, -0.00313266,  0.04329148], dtype=float32), array([ 88.,  36., 138., 104.,   2.]))
0.9795918367346939
0.930117501546073
0.9334660221779926
0.7315214630429261
0.8729641693811075
0.9209508015478165
0.8642186165670367
0.9558823529411765
0.8936530324400564
0.9004861309694023
(10, 11)
('is fg', array([0.9697789 , 0.03022103], dtype=float32))
('is fg', array([0.9

(array([ 0.03199921,  0.01218974, -0.03009883,  0.00360645], dtype=float32), array([ 87.,  35., 138., 106.,   2.]))
0.9803921568627451
0.9859154929577465
0.98
0.9400544959128065
0.9480626545754328
(5, 9)
('is fg', array([0.9978193 , 0.00218067], dtype=float32))
('is fg', array([0.9875916 , 0.01240837], dtype=float32))
('is fg', array([0.99737346, 0.00262656], dtype=float32))
('is fg', array([0.9641449 , 0.03585517], dtype=float32))
('is fg', array([0.9872804 , 0.01271953], dtype=float32))
(0.059778377, 0.00017869283)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
(array([ 0.0016123 , -0.00338649,  0.01935818, -0.00906629], dtype=float32), array([150.,  37., 199., 104.,   2.]))
0.9178368688377652
0.8012519561815337
0.9322235434007135
0.7944507361268404
0.7993079584775087
0.850246845858475
0.9066666666666666
0.946236559139785
0.8951069440375037
0.8148148148148148
(10, 9)
('is fg', array([0.93581516, 0.06418476], dtype=float32))
('is fg', array([0.97631466, 0.02368532], 

(array([ 0.00577258,  0.00258679, -0.0123645 ,  0.02553017], dtype=float32), array([149., 118., 200., 187.,   2.]))
0.9292874559739908
0.8976897689768977
0.902915291529153
0.9124423963133641
0.9042524739235089
0.9345794392523364
0.948509485094851
0.9395827688973178
(8, 15)
('is fg', array([0.9879693, 0.0120307], dtype=float32))
('is fg', array([0.9931566 , 0.00684341], dtype=float32))
('is fg', array([0.99326986, 0.00673014], dtype=float32))
('is fg', array([0.98835164, 0.01164833], dtype=float32))
('is fg', array([0.995484  , 0.00451603], dtype=float32))
('is fg', array([0.9970055 , 0.00299444], dtype=float32))
('is fg', array([0.9044132 , 0.09558681], dtype=float32))
('is fg', array([0.965164  , 0.03483604], dtype=float32))
(0.022982858, 0.00060292386)
-----
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
(array([-0.01561795,  0.01534182,  0.01250567, -0.00470399], dtype=float32), array([151., 120., 199., 187.,   2.]))
0.9545454545454546
0.9518315546582059
0.92307692307

In [ ]:
import tensorflow as tf
tf.gather?